<a href="https://colab.research.google.com/github/RivaroFarrelino/Fake-News-Detection/blob/main/FakeNewsDetection_Project_Akhir_DataSains.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Anggota Kelompok**
1. Muhammad Rivaro Farrelino Gozali - 225150707111076
2. Dwi Rama Prasetiya - 225150700111036
3. Muhammad Rifqi Taufan - 225150707111080

### Importing Libraries and Dataset
Library yang digunakan

- NumPy: Untuk function matematika
- Pandas: Untuk load dataset.
- Tensorflow: Untuk preprocessing data dan untuk membuat model
- SkLearn: Untuk train-test split dan untuk import modul untuk model evaluation.



In [ ]:
!pip install tensorflow

In [ ]:
import numpy as np
import pandas as pd
import json
import csv
import random
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers

import pprint
from tensorflow.python.framework import ops
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

# Reading the data
data = pd.read_csv("news.csv")
data.head()

data.head(5)


,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


### Preprocessing Dataset

In [ ]:
data = data.drop(["Unnamed: 0"], axis=1)
data.head(5)

,title,text,label
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


### Data Encoding
mengonversi kolom kategorikal (label dalam kasus kami) menjadi nilai numerik.

In [ ]:
# encoding the labels
le = preprocessing.LabelEncoder()
le.fit(data['label'])
data['label'] = le.transform(data['label'])

In [ ]:
embedding_dim = 50
max_length = 54
trunc_type = 'post'
padding_type = 'post'
oov_tok = "<OOV>"
training_size = 3000
test_portion = .1

###Tokenization
Proses ini membagi teks kontinu yang besar menjadi unit atau token yang terpisah pada dasarnya. Di sini, kami menggunakan kolom secara terpisah untuk dasar temporal sebagai pipeline demi akurasi yang lebih baik.

In [ ]:
title = []
text = []
labels = []
for x in range(training_size):
    title.append(data['title'][x])
    text.append(data['text'][x])
    labels.append(data['label'][x])

### Applying Tokenization

In [ ]:
tokenizer1 = Tokenizer()
tokenizer1.fit_on_texts(title)
word_index1 = tokenizer1.word_index
vocab_size1 = len(word_index1)
sequences1 = tokenizer1.texts_to_sequences(title)
padded1 = pad_sequences(
    sequences1,  padding=padding_type, truncating=trunc_type)
split = int(test_portion * training_size)
training_sequences1 = padded1[split:training_size]
test_sequences1 = padded1[0:split]
test_labels = labels[0:split]
training_labels = labels[split:training_size]

### Generating Word Embedding

Disini memungkinkan kata-kata dengan makna yang mirip memiliki representasi yang serupa. Di sini, setiap kata individual direpresentasikan sebagai vektor bernilai real dalam ruang vektor yang telah ditentukan sebelumnya. Untuk itu, kami akan menggunakan glove.6B.50d.txt. File ini memiliki ruang vektor yang telah ditentukan untuk kata-kata.



In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip # Download GloVe embeddings
!unzip glove.6B.zip                            # Unzip the downloaded file

embeddings_index = {}
with open('glove.6B.50d.txt') as f:            # Now open the file
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

# Generating embeddings
embeddings_matrix = np.zeros((vocab_size1+1, embedding_dim))
for word, i in word_index1.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embeddings_matrix[i] = embedding_vector

# This code is modified by Susobhan Akhuli

--2024-12-03 09:43:35--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2024-12-03 09:43:35--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2024-12-03 09:43:35--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.2’

gl

### Creating Model Architecture

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size1+1, embedding_dim,
                              input_length=max_length, weights=[
                                  embeddings_matrix],
                              trainable=False),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv1D(64, 5, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=4),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',
              optimizer='adam', metrics=['accuracy'])
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │         377,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d (Conv1D)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d (MaxPooling1D)         │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 377,600 (1.44 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 377,600 (1.44 MB)

In [ ]:
num_epochs = 50

training_padded = np.array(training_sequences1)
training_labels = np.array(training_labels)
testing_padded = np.array(test_sequences1)
testing_labels = np.array(test_labels)

history = model.fit(training_padded, training_labels, epochs=num_epochs, validation_data=(testing_padded, testing_labels), verbose=2)

print("Training Complete")

Epoch 1/50
85/85 - 4s - 51ms/step - accuracy: 0.6322 - loss: 0.6437 - val_accuracy: 0.6833 - val_loss: 0.5737
Epoch 2/50
85/85 - 2s - 26ms/step - accuracy: 0.7037 - loss: 0.5706 - val_accuracy: 0.7167 - val_loss: 0.5276
Epoch 3/50
85/85 - 1s - 16ms/step - accuracy: 0.7248 - loss: 0.5342 - val_accuracy: 0.7067 - val_loss: 0.5116
Epoch 4/50
85/85 - 2s - 26ms/step - accuracy: 0.7733 - loss: 0.4770 - val_accuracy: 0.7133 - val_loss: 0.5356
Epoch 5/50
85/85 - 2s - 26ms/step - accuracy: 0.7996 - loss: 0.4401 - val_accuracy: 0.7100 - val_loss: 0.5755
Epoch 6/50
85/85 - 3s - 32ms/step - accuracy: 0.8137 - loss: 0.4067 - val_accuracy: 0.7433 - val_loss: 0.4999
Epoch 7/50
85/85 - 3s - 34ms/step - accuracy: 0.8404 - loss: 0.3600 - val_accuracy: 0.7567 - val_loss: 0.4973
Epoch 8/50
85/85 - 5s - 60ms/step - accuracy: 0.8637 - loss: 0.3201 - val_accuracy: 0.7267 - val_loss: 0.5644
Epoch 9/50
85/85 - 3s - 30ms/step - accuracy: 0.8811 - loss: 0.2830 - val_accuracy: 0.7367 - val_loss: 0.6180
Epoch 10/5

### Model Evaluation and Prediction
Detection model dibuat menggunakan TensorFlow

In [ ]:
# sample text to check if fake or not
X = "The Battle of New York: Why This Primary Matters"

# detection
sequences = tokenizer1.texts_to_sequences([X])[0]
sequences = pad_sequences([sequences], maxlen=54,
                          padding=padding_type,
                          truncating=trunc_type)
if(model.predict(sequences, verbose=0)[0][0] >= 0.5):
    print("This news is True")
else:
    print("This news is false")

This news is True
